# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [2]:
import pandas as pd 

# load the data 
data =  pd.read_csv('homepage_actions.csv')
data.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [3]:
# Hypothesis 
#null hypothesis
H_0 = "The proportion of viewers who clicked on the homepage in the experiment group is equal to the proportion in the control group."

# alternative hypothesis 
H_1 = "The proportion of viewers  who clicked on the homepage in the experiment group is greater than the proportion in the control group"

print("Null Hypothesis;", H_0)
print("Alternative Hypothesis;", H_1)

Null Hypothesis; The proportion of viewers who clicked on the homepage in the experiment group is equal to the proportion in the control group.
Alternative Hypothesis; The proportion of viewers  who clicked on the homepage in the experiment group is greater than the proportion in the control group


In [4]:
import numpy as np
import statsmodels.api as sm

# Calculate the number of viewers who clicked in the experiment and control groups
experiment_clicked = data[(data['group'] == 'experiment') & (data['action'] == 'click')]['id'].nunique()
control_clicked = data[(data['group'] == 'control') & (data['action'] == 'click')]['id'].nunique()

# Calculate the total number of viewers in the experiment and control groups
experiment_viewed = data[data['group'] == 'experiment']['id'].nunique()
control_viewed = data[data['group'] == 'control']['id'].nunique()

# Calculate the proportions of viewers who clicked in each group
prop_experiment_clicked = experiment_clicked / experiment_viewed
prop_control_clicked = control_clicked / control_viewed

# Perform a Z-test for two proportions
z_score, p_value = sm.stats.proportions_ztest([experiment_clicked, control_clicked], [experiment_viewed, control_viewed], alternative='larger')

# Check if the p-value is less than the significance level
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis. The experimental homepage is more effective.")
else:
    print("Fail to reject the null hypothesis. There is no significant difference in homepage effectiveness.")

# Print the test statistic and p-value
print(f"Z-Score: {z_score}")
print(f"P-Value: {p_value}")


Reject the null hypothesis. The experimental homepage is more effective.
Z-Score: 2.618563885349469
P-Value: 0.004415037788297902


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [5]:
# Calculate the expected proportion of clicks for the experiment group (assuming the same click-through rate as control)
expected_prop_experiment_clicked = prop_control_clicked

# Calculate the number of viewers in the experiment group
experiment_viewed = data[data['group'] == 'experiment']['id'].nunique()

# Calculate the expected number of clicks in the experiment group
expected_experiment_clicked = expected_prop_experiment_clicked * experiment_viewed

print(f"Expected Clicks in Experiment Group: {expected_experiment_clicked}")


Expected Clicks in Experiment Group: 838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [6]:
import math

# Calculate the standard deviation of the number of clicks
std_dev_experiment_clicked = math.sqrt(experiment_viewed * expected_prop_experiment_clicked * (1 - expected_prop_experiment_clicked))

# Calculate the actual number of clicks in the experiment group
actual_experiment_clicked = experiment_clicked

# Calculate the z-score
z_score = (actual_experiment_clicked - expected_experiment_clicked) / std_dev_experiment_clicked

print(f"Number of Standard Deviations from Estimate: {z_score}")


Number of Standard Deviations from Estimate: 3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [7]:
import scipy.stats as stats

# Calculate the two-tailed p-value
p_value = 1 - stats.norm.cdf(abs(z_score))

print(f"P-Value: {p_value}")


P-Value: 0.00012486528006949715


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **The p-values iare less than a significance level of 0.05 which suggests that there is a strong evidence to reject the null hypothesis.**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.